In [47]:
import cv2 as cv
import numpy as np
import time

%matplotlib inline
import matplotlib.pyplot as plt

In [23]:
#Getting all labesl from coco names
path = r"C:\Users\Fernanda Januar\yolov3\data\coco.names"
with open(path) as f:
    labels = [i.strip() for i in f]
f.close()

#Loading the YoloV3 Network
cfg = r"C:\Users\Fernanda Januar\yolov3\cfg\yolov3.cfg"
weight = r"C:\Users\Fernanda Januar\yolov3\yolov3.weights"
network = cv.dnn.readNetFromDarknet(cfg, weight)

#Getting all name layers from network
all_layer_names = network.getLayerNames()

#Getting only output layers
output_layer_names = [all_layer_names[i-1] for i in network.getUnconnectedOutLayers()]

#To eliminate the weak predictions
prob_minimum = 0.5

#To eliminate the weak of bounding boxes (NMS)
threshold = 0.3

In [54]:
#RealTime Detection using openCV
cv.namedWindow("Detections")
camera = cv.VideoCapture(0)

f = 0
t = 0
while True:
    ret, frame = camera.read()
    
    #Getting blob from frame
    blob = cv.dnn.blobFromImage(frame, scalefactor=1/255.0, size=(416,416), swapRB=True, crop=False)
    
    #Implementing Network
    network.setInput(blob)
    start = time.time()
    outputs = network.forward(output_layer_names)
    end = time.time()
    
    #Calculate frame and time
    f += 1
    print("Frame - {0}, took {1:.3f} seconds".format(f, end-start))
    t += end-start
    
    #Getting bounding boxes, confidences, class number
    bounding_boxes = []
    confidences = []
    class_numbers = []
    h,w = frame.shape[:2]
    
    for output in outputs:
        for i in output:
            scores_class = i[5:]
            class_idx = np.argmax(scores_class)
            confidence_current = i[4]
            
            if confidence_current > prob_minimum:
                bounding_boxes_current = i[:4] * np.array([w,h,w,h])
                x_c, y_c, width, heigth = bounding_boxes_current
                x_min = int(x_c - (width/2))
                y_min = int(y_c - (heigth/2))
                
                #Adding result to list
                bounding_boxes.append([x_min, y_min, int(width), int(heigth)])
                confidences.append(confidence_current)
                class_numbers.append(class_idx)
                
    #Implementing NMS
    NMS_results = cv.dnn.NMSBoxes(bounding_boxes, confidences, prob_minimum, threshold)
    
    #Drawing bounding boxes & Labels
    
    for i in NMS_results:
        #Drawing bounding boxes
        x_min, y_min = bounding_boxes[i][:2]
        width, heigth = bounding_boxes[i][2:]

        colours = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")
        colour_box = colours[class_numbers[i]].tolist()

        cv.rectangle(frame, (x_min, y_min), (x_min+width, y_min+heigth), colour_box, 2)

        #Put Labels
        text = "{}".format(labels[class_numbers[i]])
        cv.putText(frame, text, (x_min-5, y_min-10), cv.FONT_HERSHEY_DUPLEX, 1.0, colour_box, 2)
        
        #Put Text
        text1 = "Frame - {0}, took {1:.3f} seconds".format(f, end-start)
        text2 = "Total amount of time {:.3f} seconds".format(t)
        
        cv.putText(frame, text1, (10, 20), cv.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255), 1)
        cv.putText(frame, text2, (10, 40), cv.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255), 1)
    
    cv.imshow("Detections", frame)
    if cv.waitKey(1) & 0xff==ord("q"):
        break
        

camera.release()
cv.destroyAllWindows()

print("\nTotal number of Frame: ", f)
print("Total amount of time {:.3f} seconds".format(t))
print("FPS: {}".format(round((f/t), 2)))


Frame - 1, took 0.334 seconds
Frame - 2, took 0.320 seconds
Frame - 3, took 0.324 seconds
Frame - 4, took 0.329 seconds
Frame - 5, took 0.344 seconds
Frame - 6, took 0.340 seconds
Frame - 7, took 0.324 seconds
Frame - 8, took 0.318 seconds
Frame - 9, took 0.319 seconds
Frame - 10, took 0.351 seconds
Frame - 11, took 0.352 seconds
Frame - 12, took 0.329 seconds
Frame - 13, took 0.322 seconds
Frame - 14, took 0.316 seconds
Frame - 15, took 0.339 seconds
Frame - 16, took 0.343 seconds
Frame - 17, took 0.335 seconds
Frame - 18, took 0.320 seconds
Frame - 19, took 0.322 seconds
Frame - 20, took 0.330 seconds
Frame - 21, took 0.352 seconds
Frame - 22, took 0.348 seconds
Frame - 23, took 0.321 seconds
Frame - 24, took 0.323 seconds
Frame - 25, took 0.316 seconds
Frame - 26, took 0.355 seconds
Frame - 27, took 0.359 seconds
Frame - 28, took 0.345 seconds
Frame - 29, took 0.314 seconds
Frame - 30, took 0.320 seconds
Frame - 31, took 0.350 seconds
Frame - 32, took 0.378 seconds
Frame - 33, took 